In [35]:
#!pip install tensorflow-gpu==2.0.0-rc0 

# Evaluation of selected best model - LEM-TF-SEQ-E7

In [104]:
import pickle
import os
import numpy as np
import tensorflow.keras as K
from scipy.sparse import hstack
from sklearn.preprocessing import normalize as scikit_normalize
from sklearn.metrics import classification_report
from evaluation import rmse_report
%matplotlib inline

In [37]:
DATASET='/home/kvassay/data/z/data/reviews_train_test_dev1_{}.pickle'
TYPE='lem_tok'
MODEL_FOLDER='/home/kvassay/data/z/models/best_model/'

## Load data

In [38]:
%%time
with open(DATASET.format(TYPE),'rb') as f:
    _,_,test_ds=pickle.load(f)

CPU times: user 7.98 s, sys: 1.22 s, total: 9.2 s
Wall time: 8.96 s


## Load model

In [93]:
def penalized_loss(y_true, y_pred):
    return K.backend.mean(K.backend.square(K.backend.abs(y_true - y_pred))/y_true)


class SentimentPredictionModel:
    def __init__(self,model_folder):
        with open(os.path.join(model_folder, 'vectorizer_summary.pickle'),'rb') as f:
            self.vectorizer_summary=pickle.load(f)
        with open(os.path.join(model_folder,'vectorizer_text.pickle'), 'rb') as f:
            self.vectorizer_text=pickle.load(f)
        self.model = K.models.load_model(os.path.join(model_folder,'keras_regressor.h5'),
                                          custom_objects={'penalized_loss': penalized_loss})
        
    @staticmethod
    def _tf_predict(vectorizer,dataset,key):
        features=vectorizer.transform([' '.join(x[key]) for x in dataset])
        return features

    def _extract_features(self,dataset,key_summary,key_text):
        summ_vecs=self._tf_predict(self.vectorizer_summary,dataset, key_summary)
        text_vecs=self._tf_predict(self.vectorizer_text,dataset, key_text)
        return scikit_normalize(hstack([summ_vecs, text_vecs],format='csr'))

    @staticmethod
    def _fix_ratings_over_limit(y_pred,cast_f=float):
        for i in range(y_pred.shape[0]):
            # fix values over limit (>5, <1)
            if y_pred[i]>5:
                y_pred[i]=cast_f(5)
            if y_pred[i]<1:
                y_pred[i]=cast_f(1)
        return y_pred
    
    def predict(self, dataset_tokenized, key_summary='summary',key_text='text',fix_overlimit=False,
                integer=False):
        X_pred=self._extract_features(dataset_tokenized,key_summary,key_text)
        y_pred=self.model.predict(X_pred.todense())
        if integer:
            y_pred= np.rint(y_pred)
            if fix_overlimit:
                y_pred=self._fix_ratings_over_limit(y_pred,cast_f=int)
        else:
            if fix_overlimit:
                y_pred=self._fix_ratings_over_limit(y_pred,cast_f=float)
        return y_pred

In [94]:
%%time
sentiment_analyzer=SentimentPredictionModel(MODEL_FOLDER)

CPU times: user 2 s, sys: 596 ms, total: 2.6 s
Wall time: 2.56 s


# Evaluation

#### Prepare predictions

In [95]:
y_test=np.array([x['score'] for x in test_ds])
y_test_int=np.array([int(x) for x in y_pred])
len(y_test)

8527

In [96]:
%%time
y_pred_raw=sentiment_analyzer.predict(test_ds,fix_overlimit=False)
y_pred_float=sentiment_analyzer.predict(test_ds,fix_overlimit=True)
y_pred_int=sentiment_analyzer.predict(test_ds,fix_overlimit=True,integer=True)

CPU times: user 22.4 s, sys: 12.2 s, total: 34.6 s
Wall time: 25.5 s


# A) - Raw output RMSE 
- we calculate RMSE scores from raw outputs of the model
- fair for comparison with development set evaluation scores
- model generalizes very well

In [100]:
rmse_report(y_test,y_pred_raw)

RMSE (baseline ∀1.0),1.533
RMSE,0.646
Mean partial RMSE (baseline ∀1.0),2.0
Max partial RMSE (baseline ∀1.0),1.414
St.dev. partial RMSE (baseline ∀1.0),4.0
Mean partial RMSE,0.683
Max partial RMSE,0.065
St.dev. partial RMSE,0.755
RMSE,0.887
Mean partial RMSE,1.317
Max partial RMSE,3.245


## B) - RMSE scores with overlimit correction
- values >5 and <1 will be replaced by 5 or 1
- evaluation of score for realistic usage

In [103]:
rmse_report(y_test,y_pred_float)

RMSE (baseline ∀1.0),1.533
RMSE,0.617
Mean partial RMSE (baseline ∀1.0),2.0
Max partial RMSE (baseline ∀1.0),1.414
St.dev. partial RMSE (baseline ∀1.0),4.0
Mean partial RMSE,0.664
Max partial RMSE,0.077
St.dev. partial RMSE,0.741
RMSE,0.916
Mean partial RMSE,1.336
Max partial RMSE,3.259


## C) - Evaluation of classification scores
- in case we wanted to use this model as Amazon review classifier
- not working so well
- categories are easy to miss however, except for 1 and 5
- possible human agreement also low

In [110]:
print(classification_report(y_test_int, y_pred_int))

              precision    recall  f1-score   support

           1       1.00      0.73      0.84       935
           2       0.56      0.54      0.55       609
           3       0.63      0.43      0.51      1116
           4       0.67      0.36      0.46      3670
           5       0.48      1.00      0.65      2197

    accuracy                           0.59      8527
   macro avg       0.67      0.61      0.60      8527
weighted avg       0.65      0.59      0.57      8527



## D) - Evaluation of classification scores - binary problem
- let's say we try to classify whether customer was satisfied
- score 1/2 => satisfied, score 3/4/6 => not satisfied
- sentiment analyser is pretty good at distinguishing between positive/negative

In [114]:
y_test_bin=np.array([1 if x <3 else 0 for x in y_test])
y_pred_bin=np.array([1 if x <3 else 0 for x in y_pred_raw])

In [115]:
print(classification_report(y_test_bin,y_pred_bin))

              precision    recall  f1-score   support

           0       0.99      0.94      0.97      7308
           1       0.74      0.93      0.82      1219

    accuracy                           0.94      8527
   macro avg       0.86      0.94      0.89      8527
weighted avg       0.95      0.94      0.95      8527

